<a href="https://colab.research.google.com/github/VaradBelwalkar/NLP_Seq2Seq/blob/main/ENG_FRE.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

## English to French Seq2Seq model

In [125]:
from keras.models import Model
from keras.layers import LSTM
from keras.layers import Dense, Input, Embedding
from keras.preprocessing.sequence import pad_sequences
from collections import Counter
from keras.callbacks import ModelCheckpoint
import nltk
import numpy as np
import os
import zipfile
import sys
import urllib.request
from gensim.models import KeyedVectors
from keras.utils import plot_model
from nltk.translate.bleu_score import sentence_bleu, SmoothingFunction
from keras.optimizers import Adam

In [127]:
from keras.activations import softmax
from keras.layers import Dense, Activation, RepeatVector, Permute
from keras.layers import Input, Embedding, Multiply, Concatenate, Lambda
from keras.layers import TimeDistributed

#hyperparameters

In [128]:
BATCH_SIZE = 64
NUM_EPOCHS = 100
HIDDEN_UNITS = 256
NUM_SAMPLES = 10000
MAX_VOCAB_SIZE = 10000
EMBEDDING_SIZE = 100
DATA = 'fra.txt'

In [129]:
tar_count = Counter()

GLOVE_MODEL = "glove.6B.100d.txt"
WEIGHT_FILE_PATH = 'eng-to-fr-glove-weights.h5'


## Downloading the GloVe embeddings zip

In [5]:
!wget http://nlp.stanford.edu/data/glove.6B.zip

--2024-06-08 09:16:21--  http://nlp.stanford.edu/data/glove.6B.zip
Resolving nlp.stanford.edu (nlp.stanford.edu)... 171.64.67.140
Connecting to nlp.stanford.edu (nlp.stanford.edu)|171.64.67.140|:80... connected.
HTTP request sent, awaiting response... 302 Found
Location: https://nlp.stanford.edu/data/glove.6B.zip [following]
--2024-06-08 09:16:21--  https://nlp.stanford.edu/data/glove.6B.zip
Connecting to nlp.stanford.edu (nlp.stanford.edu)|171.64.67.140|:443... connected.
HTTP request sent, awaiting response... 301 Moved Permanently
Location: https://downloads.cs.stanford.edu/nlp/data/glove.6B.zip [following]
--2024-06-08 09:16:21--  https://downloads.cs.stanford.edu/nlp/data/glove.6B.zip
Resolving downloads.cs.stanford.edu (downloads.cs.stanford.edu)... 171.64.64.22
Connecting to downloads.cs.stanford.edu (downloads.cs.stanford.edu)|171.64.64.22|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 862182613 (822M) [application/zip]
Saving to: ‘glove.6B.zip’

glov

## Extracting the zip to get required embeddings

In [6]:
!unzip glove.6B.zip

Archive:  glove.6B.zip
  inflating: glove.6B.50d.txt        
  inflating: glove.6B.100d.txt       
  inflating: glove.6B.200d.txt       
  inflating: glove.6B.300d.txt       


## Sample word embeddings

In [69]:
!head glove.6B.100d.txt

the -0.038194 -0.24487 0.72812 -0.39961 0.083172 0.043953 -0.39141 0.3344 -0.57545 0.087459 0.28787 -0.06731 0.30906 -0.26384 -0.13231 -0.20757 0.33395 -0.33848 -0.31743 -0.48336 0.1464 -0.37304 0.34577 0.052041 0.44946 -0.46971 0.02628 -0.54155 -0.15518 -0.14107 -0.039722 0.28277 0.14393 0.23464 -0.31021 0.086173 0.20397 0.52624 0.17164 -0.082378 -0.71787 -0.41531 0.20335 -0.12763 0.41367 0.55187 0.57908 -0.33477 -0.36559 -0.54857 -0.062892 0.26584 0.30205 0.99775 -0.80481 -3.0243 0.01254 -0.36942 2.2167 0.72201 -0.24978 0.92136 0.034514 0.46745 1.1079 -0.19358 -0.074575 0.23353 -0.052062 -0.22044 0.057162 -0.15806 -0.30798 -0.41625 0.37972 0.15006 -0.53212 -0.2055 -1.2526 0.071624 0.70565 0.49744 -0.42063 0.26148 -1.538 -0.30223 -0.073438 -0.28312 0.37104 -0.25217 0.016215 -0.017099 -0.38984 0.87424 -0.72569 -0.51058 -0.52028 -0.1459 0.8278 0.27062
, -0.10767 0.11053 0.59812 -0.54361 0.67396 0.10663 0.038867 0.35481 0.06351 -0.094189 0.15786 -0.81665 0.14172 0.21939 0.58505 -0.52158 

## Loading GloVe Embeddings

In [130]:
def load_glove():

    _word2em = {}
    file = open(GLOVE_MODEL, mode='r', encoding='utf8')
    for line in file:
        words = line.strip().split()
        word = words[0]
        embeds = np.array(words[1:], dtype=np.float32)
        _word2em[word] = embeds
    file.close()
    return _word2em

In [131]:
word2em = load_glove()

## Contents of the fra.txt

In [132]:
!head fra.txt

Go.	Va !
Run!	Cours !
Run!	Courez !
Wow!	Ça alors !
Fire!	Au feu !
Help!	À l'aide !
Jump.	Saute.
Stop!	Ça suffit !
Stop!	Stop !
Stop!	Arrête-toi !


In [133]:
nltk.download('punkt')

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!


True

## Preparing Data for Sequence Translation

In [135]:
lines = open(DATA, 'r', encoding='utf8').read().split('\n')
for line in lines[: len(lines)-1]:
    input_text, target_text = line.split('\t')
    input_words = [w for w in nltk.word_tokenize(input_text.lower())]
    target_text = '\t' + target_text + '\n'
    for char in target_text:
        tar_count[char] += 1


## Creating Token Index Dictionaries

In [136]:
target_word2idx = dict()

for idx, word in enumerate(tar_count.most_common(MAX_VOCAB_SIZE)):
    #print(word)
    target_word2idx[word[0]] = idx

target_idx2word = dict([(idx, word) for word, idx in target_word2idx.items()])

num_decoder_tokens = len(target_idx2word)

In [137]:
unknown_emb = np.random.randn(EMBEDDING_SIZE)

In [138]:
encoder_max_seq_length = 0
decoder_max_seq_length = 0

input_texts_word2em = []


## Processing Input and Target Texts

In [139]:
lines = open(DATA, 'r', encoding='utf8').read().split('\n')
for line in lines[: min(NUM_SAMPLES, len(lines)-1)]:
    input_text, target_text = line.split('\t')
    target_text = '\t' + target_text + '\n'
    input_words = [w for w in nltk.word_tokenize(input_text.lower())]
    encoder_input_wids = []
    for w in input_words:
        em = unknown_emb
        if w in word2em:
            em = word2em[w]
        encoder_input_wids.append(em)
    input_texts_word2em.append(encoder_input_wids)
    encoder_max_seq_length = max(len(encoder_input_wids), encoder_max_seq_length)
    decoder_max_seq_length = max(len(target_text), decoder_max_seq_length)

In [140]:
encoder_input_data = pad_sequences(input_texts_word2em, encoder_max_seq_length)

## decoder word2index input

In [141]:
decoder_target_data = np.zeros(shape=(NUM_SAMPLES, decoder_max_seq_length, num_decoder_tokens))
decoder_input_data = np.zeros(shape=(NUM_SAMPLES, decoder_max_seq_length, num_decoder_tokens))
lines = open(DATA, 'rt', encoding='utf8').read().split('\n')
for lineIdx, line in enumerate(lines[: min(NUM_SAMPLES, len(lines)-1)]):
    _, target = line.split('\t')
    target = '\t' + target + '\n'
    for idx, char in enumerate(target):
        if char in target_word2idx:
            w2idx = target_word2idx[char]
            decoder_input_data[lineIdx, idx, w2idx] = 1
            if idx > 0:
                decoder_target_data[lineIdx, idx-1, w2idx] = 1


In [142]:
context = dict()
context['num_decoder_tokens'] = num_decoder_tokens
context['encoder_max_seq_length'] = encoder_max_seq_length
context['decoder_max_seq_length'] = decoder_max_seq_length


## defining Encoder- Decoder Model

In [143]:
encoder_inputs = Input(shape=(None, EMBEDDING_SIZE), name='encoder_inputs')
encoder_lstm = LSTM(units=HIDDEN_UNITS, return_state=True, name='encoder_lstm')
encoder_outputs, encoder_state_h, encoder_state_c = encoder_lstm(encoder_inputs)
encoder_states = [encoder_state_h, encoder_state_c]

decoder_inputs = Input(shape=(None, num_decoder_tokens), name='decoder_inputs')
decoder_lstm = LSTM(units=HIDDEN_UNITS, return_state=True, return_sequences=True, name='decoder_lstm')
decoder_outputs, decoder_state_h, decoder_state_c = decoder_lstm(decoder_inputs,
                                                                 initial_state=encoder_states)
decoder_dense = Dense(units=num_decoder_tokens, activation='softmax', name='decoder_dense')
decoder_outputs = decoder_dense(decoder_outputs)

In [144]:
model = Model([encoder_inputs, decoder_inputs], decoder_outputs)

model.compile(loss='categorical_crossentropy', optimizer='adam')


checkpoint = ModelCheckpoint(filepath=WEIGHT_FILE_PATH, save_best_only=True)
model.fit([encoder_input_data, decoder_input_data], decoder_target_data, batch_size=BATCH_SIZE, epochs=NUM_EPOCHS,
          verbose=1, validation_split=0.3, callbacks=[checkpoint])

model.save_weights(WEIGHT_FILE_PATH)

Epoch 1/100
110/110 [==============================] - 67s 578ms/step - loss: 1.0569 - val_loss: 1.1666
Epoch 2/100


/usr/local/lib/python3.10/dist-packages/keras/src/engine/training.py:3103: UserWarning: You are saving your model as an HDF5 file via `model.save()`. This file format is considered legacy. We recommend using instead the native Keras format, e.g. `model.save('my_model.keras')`.
  saving_api.save_model(


110/110 [==============================] - 67s 612ms/step - loss: 0.9791 - val_loss: 1.1310
Epoch 3/100
110/110 [==============================] - 55s 503ms/step - loss: 0.9455 - val_loss: 1.0936
Epoch 4/100
110/110 [==============================] - 62s 566ms/step - loss: 0.9153 - val_loss: 1.0708
Epoch 5/100
110/110 [==============================] - 58s 533ms/step - loss: 0.8834 - val_loss: 1.0299
Epoch 6/100
110/110 [==============================] - 68s 618ms/step - loss: 0.8509 - val_loss: 0.9855
Epoch 7/100
110/110 [==============================] - 60s 543ms/step - loss: 0.8044 - val_loss: 0.9293
Epoch 8/100
110/110 [==============================] - 54s 497ms/step - loss: 0.7636 - val_loss: 0.8927
Epoch 9/100
110/110 [==============================] - 66s 602ms/step - loss: 0.7273 - val_loss: 0.8538
Epoch 10/100
110/110 [==============================] - 63s 571ms/step - loss: 0.6976 - val_loss: 0.8183
Epoch 11/100
110/110 [==============================] - 54s 497ms/step - lo

In [145]:
model.summary()

Model: "model_25"
__________________________________________________________________________________________________
 Layer (type)                Output Shape                 Param #   Connected to                  
 encoder_inputs (InputLayer  [(None, None, 100)]          0         []                            
 )                                                                                                
                                                                                                  
 decoder_inputs (InputLayer  [(None, None, 116)]          0         []                            
 )                                                                                                
                                                                                                  
 encoder_lstm (LSTM)         [(None, 256),                365568    ['encoder_inputs[0][0]']      
                              (None, 256),                                                 

In [146]:
encoder_model_inf = Model(encoder_inputs, encoder_states)


decoder_state_input_h = Input(shape=(HIDDEN_UNITS,))
decoder_state_input_c = Input(shape=(HIDDEN_UNITS,))
decoder_input_states = [decoder_state_input_h, decoder_state_input_c]

decoder_out, decoder_h, decoder_c = decoder_lstm(decoder_inputs,
                                                 initial_state=decoder_input_states)

decoder_states = [decoder_h , decoder_c]

decoder_out = decoder_dense(decoder_out)

decoder_model_inf = Model(inputs=[decoder_inputs] + decoder_input_states,
                          outputs=[decoder_out] + decoder_states )

In [147]:
max_encoder_seq_length = context['encoder_max_seq_length']
max_decoder_seq_length = context['decoder_max_seq_length']
num_decoder_tokens = context['num_decoder_tokens']

In [148]:
def predict_sent(input_text):
        input_seq = []
        input_wids = []
        for word in nltk.word_tokenize(input_text.lower()):
            emb = unknown_emb
            if word in word2em:
                emb = word2em[word]
            input_wids.append(emb)
        input_seq.append(input_wids)
        input_seq = pad_sequences(input_seq, max_encoder_seq_length)
        states_value = encoder_model_inf.predict(input_seq)
        target_seq = np.zeros((1, 1,num_decoder_tokens))
        target_seq[0, 0, target_word2idx['\t']] = 1
        target_text = ''
        terminated = False
        while not terminated:
            output_tokens, h, c = decoder_model_inf.predict([target_seq] + states_value)

            sample_token_idx = np.argmax(output_tokens[0, -1, :])
            sample_word = target_idx2word[sample_token_idx]
            target_text += sample_word

            if sample_word == '\n' or len(target_text) >= max_decoder_seq_length:
                terminated = True

            target_seq = np.zeros((1, 1, num_decoder_tokens))
            target_seq[0, 0, sample_token_idx] = 1

            states_value = [h, c]
        return target_text.strip()

In [149]:
print(predict_sent('this is amazing.'))

1/1 [==============================] - 0s 79ms/step
C'est grant.


In [150]:
def calculate_bleu_score(reference, candidate):
    smoothie = SmoothingFunction().method4
    return sentence_bleu([reference], candidate, smoothing_function=smoothie)


In [151]:
input_sentence = 'This is amazing.'
predicted_sentence = predict_sent(input_sentence)
print('Predicted:', predicted_sentence)

reference_sentence = 'Ceci est incroyable.'  # Reference translation
bleu_score = calculate_bleu_score(reference_sentence.split(), predicted_sentence.split())
print('BLEU score:', bleu_score)


1/1 [==============================] - 0s 165ms/step
Predicted: C'est grant.
BLEU score: 0
